In [8]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import pandas_ta as ta
import time
# -------------------------
# 1️⃣ Setup API key
# -------------------------
API_KEY = "BU574HHXVVKEBXZ5"  # replace with your key
ts = TimeSeries(key=API_KEY, output_format='pandas')
# -------------------------
# 2️⃣ Function to fetch stock data
# -------------------------
def fetch_stock_av(symbol):
    print(f"Fetching {symbol} data from Alpha Vantage...")
    data, meta = ts.get_daily(symbol=symbol, outputsize='full')  # full ~20y daily
    data.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low','4. close':'Close','5. volume':'Volume'}, inplace=True)
    data.index = pd.to_datetime(data.index)
    data.sort_index(inplace=True)
    print(f"✅ {symbol} data fetched, {len(data)} rows")
    return data
# -------------------------
# 3️⃣ Function to add technical indicators
# -------------------------
def add_indicators(df):
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df['SMA_20'] = ta.sma(df['Close'], length=20)
    df['EMA_50'] = ta.ema(df['Close'], length=50)
    macd = ta.macd(df['Close'])
    df['MACD'] = macd['MACD_12_26_9'] if 'MACD_12_26_9' in macd.columns else macd.iloc[:,0]
    # Bollinger Bands
    bb = ta.bbands(df['Close'])
    # Dynamically assign columns
    bb_upper = [c for c in bb.columns if 'BBU' in c][0]
    bb_middle = [c for c in bb.columns if 'BBM' in c][0]
    bb_lower = [c for c in bb.columns if 'BBL' in c][0]
    df['BB_upper'] = bb[bb_upper]
    df['BB_middle'] = bb[bb_middle]
    df['BB_lower'] = bb[bb_lower]
    df.dropna(inplace=True)
    return df
# -------------------------
# 4️⃣ Fetch & save all stocks
# -------------------------
tickers = ["NVDA"]
for t in tickers:
    df = fetch_stock_av(t)
    df = add_indicators(df)
    df.to_csv(f"{t}_alpha.csv")
    print(f"✅ {t} data + indicators saved to {t}_alpha.csv\n")
    time.sleep(12)  # avoid Alpha Vantage rate limit (5 req/min)

Fetching NVDA data from Alpha Vantage...


ImportError: Can't connect to HTTPS URL because the SSL module is not available.